In [ ]:
import matplotlib as mpl
import sys
import json

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf

sys.path.append('../../codes/scripts/particles/')

mpl.rcParams['figure.dpi'] = 200


In [ ]:
import data_handler as dh
import metrics
import utils


In [ ]:
pms = ['pm1', 'pm2.5', 'pm10']
delta_mean = [-15, -10, -5, 5, 10, 15]
cols = []
for pm in pms:
    for dm in delta_mean:
        cols.append(f'{pm}_{dm:02d}')

In [ ]:
p_data = []
k_data = []

for pm in pms:
    for dm in delta_mean:
        data = np.load(f'../../projects/particle/pk/{pm}_{dm:02d}_pk_res.npy')
        p_data.append(data[:, 0].reshape(-1, 1))
        k_data.append(data[:, 1].reshape(-1, 1))

In [ ]:
p_df = pd.DataFrame(np.concatenate(p_data, axis=1), columns=cols)
k_df = pd.DataFrame(np.concatenate(k_data, axis=1), columns=cols)

In [ ]:
fig, axes = plt.subplots(3, figsize=(22, 16), sharex=True)
p_df.iloc[:, :6].plot(kind="hist", bins=100, ax=axes[0])
p_df.iloc[:, 6:12].plot(kind="hist", bins=100, ax=axes[1])
p_df.iloc[:, 12:18].plot(kind="hist", bins=100, ax=axes[2])
axes[0].set_title('PM1', fontsize=17)
axes[1].set_title('PM2.5', fontsize=17)
axes[2].set_title('PM10', fontsize=17)
axes[0].axvline(0.215055687, linestyle='--', color='b')
axes[1].axvline(0.201964166, linestyle='--', color='b')
axes[2].axvline(0.182238429, linestyle='--', color='b')
axes[2].set_xlabel("""$p'$""", fontsize=17)
fig.suptitle("""$p'$ for each PM and $\Delta PM\_out$""", fontsize=24)


In [ ]:
fig, axes = plt.subplots(3, figsize=(22, 16), sharex=True)

k_df.iloc[:, :6].plot(kind="hist", bins=100, ax=axes[0])
k_df.iloc[:, 6:12].plot(kind="hist", bins=100, ax=axes[1])
k_df.iloc[:, 12:18].plot(kind="hist", bins=100, ax=axes[2])
axes[0].set_title('PM1', fontsize=17)
axes[1].set_title('PM2.5', fontsize=17)
axes[2].set_title('PM10', fontsize=17)
axes[0].axvline(0.01387988, linestyle='--', color='b')
axes[1].axvline(0.014322877, linestyle='--', color='b')
axes[2].axvline(0.014886466, linestyle='--', color='b')
axes[2].set_xlabel("""$k'$""", fontsize=17)
fig.suptitle('''$k'$ for each PM and $\Delta PM\_out$''', fontsize=24)


## 신뢰구간 검증

In [ ]:
usable_dates = [
    {"start": "2022-05-07 09:40", "end": "2022-05-17 08:38"},
    {"start": "2022-05-17 11:25", "end": "2022-05-30 23:26"},
    {"start": "2022-06-01 22:40", "end": "2022-07-02 07:00"},
    {"start": "2022-07-02 16:40", "end": "2022-07-09 07:13"},
    {"start": "2022-07-09 14:30", "end": "2022-07-12 10:00"},
    {"start": "2022-07-25 12:00", "end": "2022-08-01 10:00"},
    {"start": "2022-08-03 09:00", "end": "2022-08-11 22:18"},
    {"start": "2022-08-12 12:14", "end": "2022-08-20 00:00"},
    {"start": "2022-08-20 09:38", "end": "2022-09-01 00:00"}
]
outputs = ['PM1', 'PM2.5', 'PM10']
inputs = [
    'PM1_2.5_OUT',
    'PM1_2.5_H_OUT',
    'PM2.5_OUT',
    'PM2.5_H_OUT',
    'PM2.5_10_OUT',
    'PM2.5_10_H_OUT',
    'PERSON_NUMBER',
    'AIR_PURIFIER',
    'WINDOW',
    'AIR_CONDITIONER',
    'DOOR',
    'TEMPERATURE',
    'WIND_SPEED',
    'WIND_DEG',
    'HUMIDITY'
]


In [ ]:
train_dfs, val_dfs, test_dfs = dh.get_datasets(
    inputs,
    outputs,
    usable_dates,
    0.1,
    0.25,
    window=20,
    method='mean'
)

df = pd.concat(train_dfs)

In [ ]:
pm1_df = df[['PM1_2.5_OUT', 'PM1_2.5_H_OUT', 'PM1', 'WIND_DEG', 'WIND_SPEED', 'HUMIDITY', 'TEMPERATURE',]].copy()
pm1_df = (pm1_df - pm1_df.mean()) / pm1_df.std()
pm2_df = df[['PM2.5_OUT', 'PM2.5_H_OUT', 'PM2.5', 'WIND_DEG', 'WIND_SPEED', 'HUMIDITY', 'TEMPERATURE',]].copy()
pm2_df = (pm2_df - pm2_df.mean()) / pm2_df.std()
pm10_df = df[['PM2.5_10_OUT', 'PM2.5_10_H_OUT', 'PM10', 'WIND_DEG', 'WIND_SPEED', 'HUMIDITY', 'TEMPERATURE',]].copy()
pm10_df = (pm10_df - pm10_df.mean()) / pm10_df.std()

In [ ]:
ax = pm1_df.cov().loc['PM1'].plot(kind='bar')
pm2_df.cov().loc['PM2.5'].plot(kind='bar', ax=ax)
pm10_df.cov().loc['PM10'].plot(kind='bar', ax=ax)

In [ ]:
pm2_df.cov()

In [ ]:
pm10_df.cov()